In [69]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data =  {
  "input_data": {
    "columns": [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22],
    "index": [0, 1],
    "data": [
            [20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0],
            [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8]
        ]
  }
}

def callCreditScoreModel(data, convert_json=True):
    if convert_json:
        data = json.dumps(data)
    body = str.encode(data)

    url = 'https://credit-defaults-model-21.eastus.inference.ml.azure.com/score'
    # Replace this with the primary/secondary key or AMLToken for the endpoint
    api_key = 'zjk4gbk2t5PeNR4QM8xz9lfWe8sBYv4d'
    if not api_key:
        raise Exception("A key should be provided to invoke the endpoint")

    # The azureml-model-deployment header will force the request to go to a specific deployment.
    # Remove this header to have the request observe the endpoint traffic rules
    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'blue' }

    req = urllib.request.Request(url, body, headers)

    try:
        response = urllib.request.urlopen(req)

        result = response.read()
        return result
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))


In [70]:
result = callCreditScoreModel(data)
print(result)


b'[1, 0]'


# Generate Synthetic Data For Data Drift

In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

import os 
from dotenv import load_dotenv

# load the environment variables from .env
load_dotenv()

# authenticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id = os.environ.get('SUBSCRIPTION_ID'),
    resource_group_name = os.environ.get('RESOURCE_GROUP_NAME'),
    workspace_name = os.environ.get('WORKSPACE_NAME'),
)

In [72]:
import pandas as pd

# get a handle of the data asset and print the URI
v1="2023.09.06.135022"
data_asset = ml_client.data.get(name="credit-card", version=v1)
print(f"Data asset URI: {data_asset.path}")

# read into pandas - note that you will see 2 headers in your data frame - that is ok, for now

df = pd.read_csv(data_asset.path, skiprows=1)
df.head()

Data asset URI: azureml://subscriptions/f1ea6ed8-82f3-416d-881b-8b376218bc85/resourcegroups/rg_aml/workspaces/aml-default/datastores/workspaceblobstore/paths/LocalUpload/4b1dfc4d12429b46389cabdf25b886a2/default_of_credit_card_clients.csv


,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [73]:
import numpy as np
import random
# Step 1: Select the column
age_column = df['AGE']

# Step 2: Apply a function to each value
def add_subtract_one_random(x):
    yr= 1 if random.random() < 0.5 else -1
    return (x+yr)

new_age_column = age_column.apply(add_subtract_one_random)
# Step 3: Assign the new values back to the column
df['AGE'] = new_age_column
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,25,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,25,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,33,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,36,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,56,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [74]:
def random_pay(x):
    if x == 2:
        return 1
    elif x == -2:
        return -1
    else:
        y= 1 if random.random() < 0.5 else -1
        return (x+y)
df['PAY_0'] = df['PAY_0'].apply(random_pay)
df['PAY_2'] = df['PAY_2'].apply(random_pay)
df['PAY_3'] = df['PAY_3'].apply(random_pay)
df['PAY_4'] = df['PAY_4'].apply(random_pay)
df['PAY_5'] = df['PAY_5'].apply(random_pay)
df['PAY_6'] = df['PAY_6'].apply(random_pay)
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,25,1,1,0,0,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,25,-2,1,-1,1,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,33,-1,1,-1,-1,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,36,-1,-1,-1,-1,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,56,0,1,0,1,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [75]:
from random import randint
def random_bill_amt(x):
    if x == 0:
        return 0
    else:
        y = randint(1, 100) 
        return (x+y)

df['BILL_AMT1'] = df['BILL_AMT1'].apply(random_bill_amt)
df['BILL_AMT2'] = df['BILL_AMT2'].apply(random_bill_amt)
df['BILL_AMT3'] = df['BILL_AMT3'].apply(random_bill_amt)
df['BILL_AMT4'] = df['BILL_AMT4'].apply(random_bill_amt)
df['BILL_AMT5'] = df['BILL_AMT5'].apply(random_bill_amt)
df['BILL_AMT6'] = df['BILL_AMT6'].apply(random_bill_amt)
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,25,1,1,0,0,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,25,-2,1,-1,1,...,3365,3538,3330,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,33,-1,1,-1,-1,...,14372,14966,15636,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,36,-1,-1,-1,-1,...,28323,28973,29645,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,56,0,1,0,1,...,20992,19185,19226,2000,36681,10000,9000,689,679,0


In [76]:
df['PAY_AMT1'] = df['PAY_AMT1'].apply(random_bill_amt)
df['PAY_AMT2'] = df['PAY_AMT2'].apply(random_bill_amt)
df['PAY_AMT3'] = df['PAY_AMT3'].apply(random_bill_amt)
df['PAY_AMT4'] = df['PAY_AMT4'].apply(random_bill_amt)
df['PAY_AMT5'] = df['PAY_AMT5'].apply(random_bill_amt)
df['PAY_AMT6'] = df['PAY_AMT6'].apply(random_bill_amt)

In [77]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,25,1,1,0,0,...,0,0,0,0,730,0,0,0,0,1
1,2,120000,2,2,2,25,-2,1,-1,1,...,3365,3538,3330,0,1021,1001,1097,0,2077,1
2,3,90000,2,2,2,33,-1,1,-1,-1,...,14372,14966,15636,1580,1521,1021,1013,1064,5087,0
3,4,50000,2,2,1,36,-1,-1,-1,-1,...,28323,28973,29645,2022,2057,1208,1195,1145,1045,0
4,5,50000,1,2,1,56,0,1,0,1,...,20992,19185,19226,2079,36695,10015,9039,763,764,0


In [78]:
data =  {
  "input_data": {
    "columns": [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22],
    "index": [0],
    "data": [
        [20000, 2, 2, 1, 23, 1, 1, -2, -2, -1, -1, 3920, 3157, 781, 0, 0, 0, 0, 746, 0, 0, 0, 0]
        ]
  }
}
callCreditScoreModel(data)

b'[0]'

In [79]:
print(df.columns)

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')


In [ ]:
import pandas as pd
import re
#df = df.reset_index()  # make sure indexes pair with number of rows
#                [row['SEX'], row['EDUCATION'], row['MARRIAGE'], row['AGE'], row['PAY_0'], row['PAY_2'], row['PAY_3'], row['PAY_4'], row['PAY_5'], row['BILL_AMT1'], row['BILL_AMT2'], row['BILL_AMT3'],row['BILL_AMT4'], row['BILL_AMT5'], row['BILL_AMT6'],row['PAY_AMT1'], row['PAY_AMT3'], row['PAY_AMT4'], row['PAY_AMT5'], row['PAY_AMT6']]
data1 = """ {
  "input_data": {
    "columns": [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22],
    "index": [0],
    "data": [
        [
    """

data2 = """
        ]
            ]
  }
} """    
 
for index, row in df.iterrows():
    data = f"""
    {row['LIMIT_BAL']},{row['SEX']},{row['EDUCATION']},{row['MARRIAGE']},{row['AGE']},{row['PAY_0']},{row['PAY_2']}, {row['PAY_3']}, {row['PAY_4']}, {row['PAY_5']}, {row['PAY_6']}, {row['BILL_AMT1']}, {row['BILL_AMT2']}, {row['BILL_AMT3']},{row['BILL_AMT4']}, {row['BILL_AMT5']}, {row['BILL_AMT6']},{row['PAY_AMT1']},{row['PAY_AMT2']}, {row['PAY_AMT3']}, {row['PAY_AMT4']}, {row['PAY_AMT5']}, {row['PAY_AMT6']}
    """
    payload= (data1 + data + data2)
    clean_payload = re.sub(r"[\n\t\s]*", "", payload)
    result = callCreditScoreModel(clean_payload,convert_json=False)

    if (index % 500) == 0:
        print(f"====={index}==============")
        print(clean_payload)
        print(result)
               
